<span style="color:red">Abgegeben von (Name, Vorname):</span> 
Goxhufi, Driton

# Vorbereitung


Wie immer müssen wir zuerst das NLTK-Modul laden. Außerdem importieren wir das Brown Corpus.

In [1]:
import nltk
from nltk.corpus import brown

## Morphologie

(Hinweis: Eine ausführlichere Kurzeinleitung gibt es in den beigestellten Folien (`skript-morphologie.pdf`).

Die Morphologie innerhalb der Linguistik beschäftigt sich mit dem **inneren Aufbau von Wortformen** anhand von Morphemen.

Es gibt verschiedene Arten von **Morphemen**:
- **Wurzel**: der kleinste Teil eines Wortes, der die zentralen semantischen Informationen beiträgt und kein Affix enthält.
    - z.B. *zer-**leg**-en*
- **Derivationsaffixe**: eine geschlossene Klasse von Morphemen, die die Wortart (Nomen, Verb, Adjektiv, ...) bestimmen können
    - z.B. **zer**-*leg-en*
- **Stamm**: der Teil eines Wortes, der die Wurzel und alle Derivationsaffixe enthält.
    - z.B. **zer-leg**-*en*
- **Flexionsaffixe**: eine geschlossene Klasse von Morphemen, die immer an letzter Stelle im Wort stehen und bestimmte semantische (Genus, Komparation, Person, Numerus, Aspekt, Aktionsart, Tempus, Modus) oder funktionale (Kasus) Informationen beitragen.
    - z.B. *zer-leg*-**en**

Unter Normalisierung im Bereich der Morphologie versteht man die Ersetzung einer Wortform durch ihren Stamm oder durch eine Zitationsform (oder Lemma).


## Warum Normalisierung?

- Wenn wir wissen wollen, wie groß der Wortschatz eines Textes ist.
- Wenn wir wissen wollen, wie oft bestimmte Begriffe in einem Text vorkommen.
    - z.B. Information Retrieval, Textklassifizierung
- Wenn wir Suchanfragen verallgemeinern möchten.
- ...

Die Normalisierung ist besonders interessant bei Sprachen mit reicher Flexion. Dazu gehört das Deutsche.

# Normalisierung der Groß-/Kleinschreibung

Die einfachste Form der Normalisierung ist die Vereinheitlichung der Groß-/Kleinschreibung.

Zum Beispiel kann man durchgängig die Kleinschreibung verwenden. Dafür steht die Standardfunktion `lower()` zur Verfügung.

In [2]:
word = "Peter went to the White House and read the New York Times."
word.lower()

'peter went to the white house and read the new york times.'

Das Problem bei dieser rasenmäherartigen Normalisierung besteht darin, dass gewisse Unterscheidungsmöglichkeiten hinsichtlich der Bedeutung verloren gehen können.

- *White House* $\Rightarrow$ *white house* 
- *Peter* $\Rightarrow$ *peter*

Besser ist es, nur das erste Wort eines Satzes klein zu schreiben $-$ aber auch nur dann, wenn das Wort sonst irgendwo im Korpus klein geschrieben wird. 

## <span style="color:red">Aufgaben I</span>

<span style="color:red">A1:</span> Schreiben Sie eine Funktion `conditional_lower(input,corpusSentsTails)` die, ein Wort `word` in Kleinschreibung ausgibt, falls `word` in `corpusSentsTails` klein geschrieben wird.

Nehmen Sie der Einfachheit halber an, dass `corpuSentsTails` diejenigen Wortformen aus dem Brown Corpus (Textsorte "news") enthält, die **nicht** satz-initial stehen.

In [50]:
# Lösung zu A1:
def conditional_lower(input, corpusSentsTails):
    lowWord = [];
    for i in corpusSentsTails[1:]:
        if input in i:
            print(i.lower())
    return i.lower();

In [51]:
noninitialSet = brown.words(categories=['news'])

In [53]:
#[conditional_lower(word,noninitialSet) for word in ["That"]]

Hier ein Testset für `conditional_lower()`:

In [54]:
[conditional_lower(word,noninitialSet) for word in ["Pizza", "Peter", "That", "Times", "White", "Urlaubsantrag"]]

petersburg
petersburg
peter
peter
peter
peter
peter
peterson
peterson
peterson
peterson
peter
peter
richmond-petersburg
that
that
that
that
that's
that
that
that's
that
that
that's
that
that
that
that
that
that
that
that
that
that
that
that
that
that's
that's
that
that
that
that
that
that
times
times-picayune
times
white
white
white
white
white
white
white
white
white
white
whitey
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white


['!', '!', '!', '!', '!', '!']

# Stemming (Rückschnitt)

Als Stemming bezeichnet man Verfahren, die die Suffixe eines Wortes tilgen und dadurch unterschiedliche Wortformen vereinheitlichen: 

- *connect* $\Rightarrow$ *connect*
- *connected* $\Rightarrow$ *connect*
- *connecting* $\Rightarrow$ *connect*
- *connection* $\Rightarrow$ *connect*
- *connections* $\Rightarrow$ *connect*

Man beachte: 
- Solche Verfahren zielen **nicht** auf eine linguistisch adäquate Bildung des Stammes! Das ist Thema der Computationellen Morphologie bzw. Ziel des morphologischen Parsens. (siehe https://aclweb.org/aclwiki/Morphology_software_for_English)
- Solche Verfahren können die Grenze zwischen Wortarten verwischen.
- Der Übergang zwischen nützlichem und schädlichem Stemming ist fließend und auch abhängig von der Anwendung.
    - *relate* $\Rightarrow$ *relate*
    - *relativity* $\Rightarrow$ *relate*    (schädlich?)
    - *organ* $\Rightarrow$ *organ*  
    - *organization* $\Rightarrow$ *organ*    (schädlich!)
    
Die Funktionsweise ist vergleichsweise simpel: Das Stemming wird von bedingten Ersetzungsregeln der Form 
- `(condition) S1 -> S2`

kontrolliert, wobei `S1` durch `S2` ersetzt wird, wenn `condition` erfüllt ist.

Die Stemmingverfahren unterscheiden sich im Prinzip darin, was in `condition`, `S1` und `S2` stehen kann, wie diese Regeln geordnet sind (das längste `S1` gewinnt!), und ob es mehrere Durchläufe gibt.

## Stemming mit regulären Ausdrücken

Die einfachsten Stemming-Methoden verwenden reguläre Ausdrücke für Affixe, die getilgt werden, falls Sie am Ende der Wortform stehen. (Dann werden die Affixe auch *Suffixe* genannt.) Die Regeln sehen also so aus:

- `(S1 steht am Ende) S1 -> ""`

Mit der String-Methode `endswith()` lässt sich das ohne Probleme umsetzen (siehe https://www.nltk.org/book/ch03.html):  

In [55]:
def stem(word):
    for suffix in ['ing', 'ly', 'ed', 'ious', 'ies', 'ive', 'es', 's', 'ment']:
         if word.endswith(suffix):
            return word[:-len(suffix)]
    return word

Dasselbe geht natürlich auch sehr einfach mit regulären Ausdrücken.

In [56]:
import re
def stem2(word):
    regexp = r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)?$'
    stem, suffix = re.findall(regexp, word)[0]
    return stem

In [57]:
[stem2(word) for word in ["connections", "is"]]

['connection', 'i']

Diese beiden einfachen Stemmer haben natürlich diverse Einschränkungen: 
- keine Mindestlänge
- nur Tilgungen (Problem: *women* $\Rightarrow$ *woman*)
- keine Mehrfachtilgung (wie beim Beispiel mit *connections* oben)

NLTK enthält ein Modul für die Entwicklung eines einfachen [RegExp-Stemmers](https://www.nltk.org/api/nltk.stem.html#module-nltk.stem.regexp) (`RegexpStemmer` in `nltk.stem`), der zumindest die Angabe einer Mindestlänge zulässt.

In [58]:
from nltk.stem import RegexpStemmer
st = RegexpStemmer('ing$|s$|e$|able$', min=4)
[st.stem(word) for word in ["connections", "is"]]

['connection', 'is']

Neben `RegexpStemmer` enthält NLTK aber auch fertige Stemmer für eine Reihe von Sprachen (darunter Arabisch!).

## Stemmer fürs Englische


Für das Englische stehen im Modul `nltk.stem` gleich drei Standard-Stemmer zu Auswahl (:

- der Porter-Stemmer (`PorterStemmer`)
    - Ein sehr früher Stemmer aus den späten 1970ern.
    - https://tartarus.org/martin/PorterStemmer/index.html
    - Regeln der Form `(condition) S1 -> S2` mit nur einem Durchlauf, aber unterschiedliche "Schritte".
- der Lancaster-Stemmer (`LancasterStemmer`)
    - rekursive Anwendung der Stemming-Regeln $\Rightarrow$ Overstemming
    - https://dl.acm.org/citation.cfm?id=101310
- der Snowball-Stemmer (`SnowballStemmer`)
    - Weiterentwicklung des Porter Stemmers
    - derzeit der Standard-Stemmer
    - verfügbar auch für andere Sprachen 
    - http://snowball.tartarus.org/texts/introduction.html


In [59]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer
pst = PorterStemmer()
lst = LancasterStemmer()
sste = SnowballStemmer("english")

In [60]:
word_list = ["connections","connectionistic","flies","friend", "friendship", "friends", "friendships","stabil","destabilize","misunderstandings","railroad","moonlight","football","women","woman","ate","at","generously"]
print("{0:20}{1:20}{2:20}{3:20}".format("Word","Porter Stemmer","Lancaster Stemmer","Sowball Stemmer"))
for word in word_list:
    print("{0:20}{1:20}{2:20}{3:20}".format(word,pst.stem(word),lst.stem(word),sste.stem(word)))

Word                Porter Stemmer      Lancaster Stemmer   Sowball Stemmer     
connections         connect             connect             connect             
connectionistic     connectionist       connect             connectionist       
flies               fli                 fli                 fli                 
friend              friend              friend              friend              
friendship          friendship          friend              friendship          
friends             friend              friend              friend              
friendships         friendship          friend              friendship          
stabil              stabil              stabl               stabil              
destabilize         destabil            dest                destabil            
misunderstandings   misunderstand       misunderstand       misunderstand       
railroad            railroad            railroad            railroad            
moonlight           moonligh

## Stemmer fürs Deutsche


NLTK enthält zwei fertige Stemmer für das Deutsche. 

- der Snowball Stemmer, den wir oben schon fürs Englische gesehen haben
- der Cistem Stemmer
    - entwickelt an der LMU München
    - http://www.cis.lmu.de/~weissweiler/cistem/

In [61]:
from nltk.stem import SnowballStemmer
from nltk.stem.cistem import Cistem
sstg = SnowballStemmer("german")
cist = Cistem()

In [62]:
word_list = ["Adler","Adlers","Adlern","aß","gab","Häuser","Hau","Absurditäten","Mäuserich"]
print("{0:20}{1:20}{2:20}".format("Word","Snowball Stemmer","Cistem Stemmer"))
for word in word_list:
    print("{0:20}{1:20}{2:20}".format(word,sstg.stem(word),cist.stem(word)))

Word                Snowball Stemmer    Cistem Stemmer      
Adler               adl                 adler               
Adlers              adl                 adler               
Adlern              adl                 adler               
aß                  ass                 ass                 
gab                 gab                 gab                 
Häuser              haus                hau                 
Hau                 hau                 hau                 
Absurditäten        absurditat          absurditat          
Mäuserich           mauserich           mauserich           


## Evaluierung

Die Evaluierung eines Stemmers läuft im Prinzip so ab: 

- Es gibt eine festgelegte **Testmenge** und eine gewünschte Ausgabe (der sogenannte **Gold-Standard**).
- Es wird mit bestimmten **Evaluationsmaßen** (Precision, Recall, F1-Measure, Accuracy) gemessen, wie nah der Stemmer für die Testmenge am Gold-Standard dran ist.
- Es wird die **Geschwindigkeit** des Stemmers gemessen.

Das grundsätzliche Problem: 
- Der Gold-Standard steht nicht wirklich fest und ist abhängig von der Anwendung und dem linguistischen Anspruch.
- Es ist prinzipiell unklar, wie man Geschindigkeit und Qualität miteinander verrechnen kann.
- Die Evaluationsergebnisse gelten nur für die Testmenge.

Siehe das [Cistem-Papier](http://www.cis.lmu.de/~weissweiler/cistem/) für ein Beispiel für die Evaluation eines Stemmers.


## <span style="color:red">Aufgaben II</span>

<span style="color:red">A2:</span> Wenden Sie den Snowball-Stemmer auf die Textsorte "news" im Brown-Korpus an, indem die Wortform jeweils durch das Resultat des Stemmers ersetzt wird!

In [70]:
# Lösung zu A2
# Snowball-Stemmer for the news category in brown dataset
news = brown.words(categories=['news'])
sste.stem(news)

AttributeError: 'ConcatenatedCorpusView' object has no attribute 'lower'

<span style="color:red">A3:</span> Was sind nun die 10 häufigsten "Wortformen" _ohne Stopwörter_ und um wieviel Prozent verringert sich die Anzahl der "Wortformen". 

In [ ]:
#Lösung zu A3

**Stopwörter** sind eine geschlossene Klasse von Funktionswörter wie *der*, *und*, *sich* uvm.
`stopwords` kann so wie `brown` mit der Funktion `words()` ausgelesen werden.

In [ ]:
from nltk.corpus import stopwords
stopwords.words()

# Lemmatisierung

Die Lemmatisierung ist eine weitere Form der Wortnormalisierung und dem Stemming sehr ähnlich. 

Unter Lemmatisierung versteht man die Abbildung einer Wortform auf ein **Lemma**. 

*Definition Lemma* (siehe Folien)
> Ein Lemma ist besteht aus allen Wortformen derselben Wortart, die aus einem Stamm mittels Flexion & Modifikation gebildet werden können.

Was wir bei der Lemmatisierung aber eigentlich wollen ist nicht eine Menge von Wortformen, sondern **eine** Wortform, die das Lemma repräsentiert, die sogenannte *Basisform*, *Zitierform* oder *Lemmaform* -- kurz: Lemma.

Es gibt bestimmte Konventionen, wie die Zitierform für ein Lemma gewählt wird. Z.B. ist üblicherweise das Lemma eines Verbs die Infinitivform und das Lemma eines Nomens die Nominativ-Singular-Form.  

Wichtig ist aber eigentlich nur, dass unterschiedliche Lemmata unterschiedliche Ziterformen haben. 

- {*house*, *houses*, *housed*, *housing*} $\Rightarrow$ <span style="font-variant: small-caps;">house.V</span>
- {*house*, *houses*} $\Rightarrow$ <span style="font-variant: small-caps;">house.N</span>

So genau nehmen es aber Lemmatisierer nicht unbedingt und verlassen sich, was die Wortart betrifft, gerne auf den POS-Tagger. 


## WordNet Lemmatizer

WordNet ist ein elektronisches Lexikon mit semantischen Informationen in Form von Begriffsnetzen. Wir werden uns diese sehr einschlägige Ressource in einer der nächsten Sitzungen genauer anschauen. 

NLTK enthält einen Lemmatisierer auf Grundlage der morphologischen Informationen in WordNet. Daher muss WordNet zunächst mit `download()` heruntergeladen werden.

In [ ]:
#nltk.download()

Einmal heruntergeladen kann der [WordNet-Lemmatisierer](https://www.nltk.org/api/nltk.stem.html#module-nltk.stem.wordnet) importiert werden.

In [ ]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

Der WordNet-Lemmatisierer normalisiert eine Eingabe, indem in der WordNet-Datenbank das dazu passende Lemma gesucht und ausgegeben wird. Wird kein passendes Lemma gefunden, wird die Eingabe unverändert wieder ausgegeben.

Unterschiede zum Snowball-Stemmer:
- wesentlich langsamer, da dabei die WordNet-Datenbank durchsucht wird
- Erkennung von **Vokalmodifikation** (*women* $\Rightarrow$ *woman*) und **Unregelmäßigkeiten** (*mice* $\Rightarrow$ *mouse*)
- Ausgabe von vollständigen Wortformen
- keine Normalisierung von unbekannten Wortformen (*flooked*)

In [ ]:
word_list = ["connections","connectionistic","better","is","flies","friendly","friendliest","destabilize","misunderstandings","football","mice","women","ate","generously","flooked"]
print("{0:20}{1:20}{2:20}".format("Word","Sowball Stemmer","WordNet Lemmatizer"))
for word in word_list:
    print("{0:20}{1:20}{2:20}".format(word,sste.stem(word),wnl.lemmatize(word)))

Standardmäßig nimmt der WordNet-Lemmatisierer an, dass die Eingabe ein Nomen ist. Dies kann mit der zweiten Option geändert werden:

- "n": Nomen
- "v": Verb
- "a": Adjektiv

In [ ]:
print("is :", wnl.lemmatize("is",pos="v"))
print("better :", wnl.lemmatize("better",pos="a"))
print("mice :", wnl.lemmatize("mice",pos="a"))

## <span style="color:red">Aufgaben III</span>

<span style="color:red">A4:</span> Lemmatisieren Sie den News-Teil des Brown Corpus mit `wnl.lemmatize()`, indem Sie abhängig vom POS-Tag eines Worttokens das `pos`-Argument von `wnl.lemmatize()` spezifizieren.

In [ ]:
# Lösung zu A4

Hinweise:
- Die POS-Tags des Brown Corpus mit Beschreibung finden Sie hier: https://en.wikipedia.org/wiki/Brown_Corpus#Part-of-speech_tags_used
- Überlegen Sie sich, welche POS-Tags zu Adjektiven und Verben gehören ...


# Normalisierung von Nicht-Standard-Worten

Für Worttoken wie Zahlen, Abkürzungen und Datumsangaben gibt es spezielle Normalisierungsregeln. Z.B. können Dezimalzahlen durch die Zahl 0 ersetzt werden, und Abkürzungen durch AAA. Der Vorteil dieser Normalisierung ist, dass der Wortschatz verringert und damit das statistische Sprachmodell für viele Aufgaben verbessert wird.